# Description

evaluation based on:
- scaling with size of input
- scaling with data size
- scaling with number of runs

- no two hotels in the same place
- the fact that the coordinates reach around earth does not matter

## get dev data

In [ ]:
import os
import sys
import shutil
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
sys.path.append("..")
from test_framework import comparison_test

In [ ]:
datadir = '../data_sources/hotels_data'
if not os.path.exists(datadir):
    os.makedirs(datadir)


put as much as you need of this to datadir

https://www.dropbox.com/sh/2ebfiuzau02ns6y/AABNPprCbIiqoS2tcVFUfm14a?dl=0

## evaluate solutions on dev data

#### create testinputs

In [ ]:
AGGED = pd.read_csv(datadir + '/agg.csv',index_col='index')

In [ ]:
def generate_input(input_size):
    return [{c:np.random.uniform(AGGED.loc['min',c],AGGED.loc['max',c]) for c in AGGED.columns}
         for _ in range(input_size)]

In [ ]:
envdir = 'env_dir'
stagedir = 'staging'

In [ ]:
MINI_DATA = [3e3,6e3,12e3]
SMALL_DATA = [3e4,6e4,30e4,60e4]
BIG_DATA = [3e6,6e6,12e6,24e6,36e6,48e6,60e6]

use_datasets = MINI_DATA
data_dicts = {int(data_size):{'datafile':datadir + '/hotels-%d.csv' % data_size,
'staging_folder':stagedir} for data_size in use_datasets}


input_sizes = [1,5]#10,20,200,1000]
reps = 5

input_set = {}
for data_name in data_dicts.keys():
    input_set[data_name] = {}
    for input_size in input_sizes:
        input_set[data_name][input_size] = [generate_input(input_size) 
                                            for _ in range(reps)]

In [ ]:
WINDOWS = False #if false it assumes unix
timedata,out_dfs = comparison_test(data_dicts,input_set,
                                   envdir,stagedir,
                                   is_windows=WINDOWS)

In [ ]:
similarity = []

for id1,solution1 in enumerate(list(out_dfs.keys())[:-1]):
    _df1 = out_dfs[solution1]
    solution2 = list(out_dfs.keys())[id1+1]
    _df2 = out_dfs[solution2]
    print('----------\n%s:\n' % solution1,_df1[_df1 != _df2].unstack().dropna())
    print('----------\n%s:\n' % solution2,_df2[_df1 != _df2].unstack().dropna())
    similarity.append({
        'solution1':solution1,
        'solution2':solution2,
        'similarity':(_df1 == _df2).mean().mean()
    })
pd.DataFrame(similarity).sort_values('similarity')

In [ ]:
timedf = pd.DataFrame(timedata).pivot_table(columns='solution',values='calc_time',
                                   index=['data_id','input_id'],aggfunc='median').reset_index()

In [ ]:
allgroups = ['data_id','input_id']
for basegroup in allgroups:
    remaingroup = [c for c in allgroups if c != basegroup]
    for _gname,_gdf in timedf.groupby(basegroup):
        _gdf.drop(basegroup,axis=1).groupby(remaingroup).median()\
        .plot(kind='bar',figsize=(10,10),title='%s - %s' % (str(basegroup),str(_gname)))